# Initial Preprocessing

Cleans up and exports the original ACP dataset files into HDF5-compressed Pandas DataFrames.

AEData:
 - Raw data must be unzipped into a directory.
 - Default path for the raw data is `data/AEdata`. This can be modified from the `Notebook` class.

ICD10: 
 - Original spreadsheet for ICD-10 (March 2021) can be found [here](https://www.health.gov.za/icd-10-master-industry-table/).
 - This should be placed in the `data/ICD10` directory. 

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from multiprocessing import Pool
import logging
from collections import defaultdict
pd.set_option("display.max_columns", None)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)


In [2]:
class Notebook:
    DATA_DIR = Path("data")
    RAW_DIR = Path("data/AEdata")

    RUN_ALL = False

    # Enable ONLY if running as a script
    MULTITHREADING = False


## Profiling Functions

In [3]:
def columns_per_dtype(df):
    col_counts = {col: df[col].value_counts() for col in df.columns}
    new_dict = defaultdict(list)
    for k, v in df.dtypes.iteritems():
        new_dict[v].append(k)
    for k in new_dict.keys():
        print(f"{k}: {new_dict[k]}")


def col_counts_topn(df, n=10):
    col_counts = {col: df[col].value_counts() for col in df.columns}
    new_dict = defaultdict(list)
    for k, v in col_counts.items():
        new_dict[v.size].append(k)
    for k in sorted(new_dict.keys())[:n]:
        print(f"{k}: {new_dict[k]}")


## Event Dates File

In [4]:
def hdf5_correct_strings(df: pd.DataFrame):
    string_cols = df.select_dtypes(include="object").columns
    df[string_cols] = df[string_cols].applymap(str)
    return df


In [5]:
def convert_eddates(df: pd.DataFrame):
    # Manual Correction: The first ArrivalDtm column has values in
    # excel-style integer format (see https://stackoverflow.com/a/65460255/7662085)
    if df.ArrivalDtm.dtype != np.dtype("datetime64[ns]"):
        df["ArrivalDtm"] = pd.to_datetime(df.ArrivalDtm, unit="D", origin="1899-12-30")

    # Iterate columns pairwise, and stack them vertically into a single DF
    return pd.DataFrame(
        np.concatenate(
            [
                df[list(_)].dropna().values
                for _ in zip(df.columns[::3], df.columns[1::3])
            ]
        ),
        columns=["AESerial", "ArrivalDtm"],
    )


def convert_mainfiledates(df: pd.DataFrame):
    return hdf5_correct_strings(df)


def convert_event_dates(infile="Each event dateDONE.xlsx", outfile="event_dates.h5"):
    result = {}

    xlsx = pd.ExcelFile(Notebook.RAW_DIR / infile)
    convert_sheets = {
        "EDDATES": convert_eddates,
        "MainFileDates": convert_mainfiledates,
    }

    for sheet, converter in convert_sheets.items():
        result[sheet] = converter(pd.read_excel(xlsx, sheet, index_col=None))

    with pd.HDFStore(Notebook.DATA_DIR / outfile) as store:
        for name, df in result.items():
            store[name] = df

    return result.values()


if Notebook.RUN_ALL:
    convert_event_dates()


## SCI File

In [6]:
def process_SCI(xlsx):
    df = xlsx.copy()

    # Drop redundant columns
    redundant = (
        [
            "Admission FY Month",
            "Year Admit",
            "Month Admit",
            "Admit Hour",
            "Admit Week",
            "Year Disch",
            "Month Disch",
            "Disch Hour",
            "Disch Week",
            "Admission Fy Year",
            "Admit Day",
            "Discharge Fy Year",
            "Disch Day",
        ]
        + ["Admission Consultant", "Last Consultant", "Area", "GP Practice", "PCT"]
        + ["Admissions Date", "Admission Date", "aLT Client GUID", "Client GUID"]
    )
    df = df.drop(redundant, axis=1)

    col_counts = {col: df[col].value_counts() for col in df.columns}

    # Drop c_ prefixed columns with no values in them
    df = df.drop(
        [
            col
            for col, count in col_counts.items()
            if count.size <= 2 and col.startswith("c_")
        ],
        axis=1,
    )

    # Replace NoNSw2d with np.NAN in all applicable columns
    df = df.replace({"NoNSw2d": np.NAN, "NoObW2D": np.NAN, "Noobw2d": np.NAN})

    # Turn these 2-value string columns into binary
    binarise = {
        "Admission Type": "Elective",
        "Admission Area": "Medical Assessment Area",
        "Discharge Area": "Assessment Area Discharge",
        "c_Nausea": "1 - Nausea present",
        "c_Vomiting": "1 - Vomiting since last round",
        **{
            _: "Yes"
            for _ in [
                "Over 7 Days",
                "Over 14 Days",
                "Care Home",
                "Died During Stay",
                "Died Within 30 Days",
            ]
        },
    }

    for col, true in binarise.items():
        df[col] = df[col].apply(true.__eq__).apply(lambda x: np.nan if x == NotImplemented else x)

    # Rename some of the binarised columns for better clarity
    df = df.rename(
        columns={
            "Admission Type": "ElectiveAdmission",
            "Admission Area": "AssessmentAreaAdmission",
            "Discharge Area": "AssessmentAreaDischarge",
            "c_Vomiting": "c_Vomiting_since_last_round",
            "Spell Discharge Date": "DischargeDateTime"
        }
    )

    # Convert NEWS dates
    datetimes = ["News CreatedWhen", "News TouchedWhen", "News AuthoredDtm"]
    df[datetimes] = df[datetimes].apply(pd.to_datetime, errors="coerce")

    # Convert blood results
    # Ignore certain non-numeric entries as they make up less than 0.001%
    numeric = [
        "Urea (serum)",
        "Sodium (serum)",
        "Potassium (serum)",
        "Creatinine",
        "pO2 (POC) Venous",
    ]
    df[numeric] = df[numeric].apply(pd.to_numeric, errors="coerce")

    # Remove spaces from column names
    df.columns = df.columns.str.replace(" ", "")

    # Drop duplicates based on serial code
    df = df.sort_values('SEQ', ascending=False).drop_duplicates("SpellSerial")

    df = df.replace('nan', np.nan)

    return df.reset_index(drop=True)


In [7]:
if Notebook.RUN_ALL or True:
    infile = Notebook.RAW_DIR / "Copy of SCI11868 Delivered 7 Ian Browne.xlsx"
    outfile = Notebook.DATA_DIR / "sci.h5"

    logging.info(f"Reading file: {infile}")
    xlsx = pd.read_excel(infile)

    logging.info(f"Processing file: {infile}")
    df = process_SCI(xlsx)

    logging.info(f"Writing to: {outfile}/table")
    df.to_hdf(outfile, key="table")


2022-07-22 13:00:58,159 [INFO] Reading file: data\AEdata\Copy of SCI11868 Delivered 7 Ian Browne.xlsx
2022-07-22 13:06:09,583 [INFO] Processing file: data\AEdata\Copy of SCI11868 Delivered 7 Ian Browne.xlsx
2022-07-22 13:06:18,308 [INFO] Writing to: data\sci.h5/table
C:\Users\stybl\AppData\Local\Temp\ipykernel_7584\3731441436.py:12: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block4_values] [items->Index(['SpellSerial', 'PatientType', 'IntendedManagement',
       'AdmissionMethodDescription', 'AdmissionSpecialty', 'LastSpecialty',
       'AdmitWard', 'NextWard2', 'NextWard3', 'NextWard4', 'NextWard5',
       'NextWard6', 'NextWard7', 'NextWard8', 'NextWard9', 'DischargeWard',
       'LOSBand', 'Gender', 'AgeBand', 'AESerial', 'AandEPresentingComplaint',
       'AandEMainDiagnosis', 'AandELocation', 'AandEPatientGroupDescription',
       'MainICD10', 'MainDiagnosis', 'SecD

## Admissions Files

In [49]:
def reconstruct_datetime(df, old, new):
    df[new] = pd.to_datetime(
        df[old]
        .dropna(how="any")
        .astype(str)
        .replace("\.0", "", regex=True)
        .apply(" ".join, 1),
        format="%Y %m %W %A %H",
    )


def process_AD(xlsx):
    df = xlsx.copy()

    # Construct DateTime from the individual columns describing admission/discharge date
    adm_dt, disch_dt = (
        ["YearAdmit", "MonthAdmit", "AdmitWeek", "AdmitDay", "AdmitHour"],
        ["YearDisch", "MonthDisch", "DischWeek", "DischDay", "DischHour"],
    )
    reconstruct_datetime(df, adm_dt, "AdmissionDateTime")
    if "SpellDischargeDate" in df.columns:
        df = df.rename(columns={"SpellDischargeDate": "DischargeDateTime"})
    else:
        reconstruct_datetime(df, disch_dt, "DischargeDateTime")

    # Turn these 2-value string columns into binary
    binarise = {
        "AdmissionType": "Elective",
        "AdmissionArea": "Medical Assessment Area",
        "DischargeArea": "Assessment Area Discharge",
        **{
            _: "Yes"
            for _ in [
                "Over7Days",
                "Over14Days",
                "CareHome",
                "DiedDuringStay",
                "DiedWithin30Days",
            ]
        },
    }

    for col, true in binarise.items():
        df[col] = df[col].apply(true.__eq__).apply(lambda x: np.nan if x == NotImplemented else x)

    # Rename some of the binarised columns for better clarity
    df = df.rename(
        columns={
            "AdmissionType": "ElectiveAdmission",
            "AdmissionArea": "AssessmentAreaAdmission",
            "DischargeArea": "AssessmentAreaDischarge",
        }
    )

    # Drop individual date component columns and some extraneous ones
    df = df.drop(
        adm_dt
        + disch_dt
        + [
            "DischargeFYear",
            "AdmissionFYear",
            "AdmissionFYMonth",
            "AdmissionConsultant",
            "LastConsultant",
            "Area",
            "PCT",
            "GPPractice",
            "AdmissionWardEndDate",
        ],
        axis=1,
        errors="ignore",
    )

    # Drop duplicates based on serial code
    df = df.drop_duplicates("SpellSerial")

    return df.reset_index(drop=True)


In [51]:
def process_AD_single(infile):
    logging.info(f"Reading file: {infile}")
    xlsx = pd.read_excel(infile)

    logging.info(f"Processing file: {infile}")
    return process_AD(xlsx)


if Notebook.RUN_ALL or True:
    indir = "AD"
    outfile = Notebook.DATA_DIR / "AD.h5"
    results = []

    if Notebook.MULTITHREADING:
        with Pool(3) as p:
            results = p.map(
                process_AD_single, list((Notebook.RAW_DIR / indir).iterdir())
            )
    else:
        for infile in (Notebook.RAW_DIR / indir).iterdir():
            results.append(process_AD_single(infile))

    logging.info(f"Writing all to {outfile}")
    r = pd.concat(results)
    r.to_hdf(outfile, key="table")


2022-07-20 09:13:38,608 [INFO] Reading file: data\AEdata\AD\Admission Data FY2014-15DONE.xlsx
2022-07-20 09:16:25,233 [INFO] Processing file: data\AEdata\AD\Admission Data FY2014-15DONE.xlsx
2022-07-20 09:16:32,573 [INFO] Reading file: data\AEdata\AD\Admission Data FY2015-16DONE.xlsx
2022-07-20 09:19:27,155 [INFO] Processing file: data\AEdata\AD\Admission Data FY2015-16DONE.xlsx
2022-07-20 09:19:31,283 [INFO] Reading file: data\AEdata\AD\Admission Data FY2016-17DONE.xlsx
2022-07-20 09:23:30,736 [INFO] Processing file: data\AEdata\AD\Admission Data FY2016-17DONE.xlsx
2022-07-20 09:23:38,861 [INFO] Reading file: data\AEdata\AD\Admission Data FY2017-18DONE.xlsx
2022-07-20 09:26:36,617 [INFO] Processing file: data\AEdata\AD\Admission Data FY2017-18DONE.xlsx
2022-07-20 09:26:42,001 [INFO] Reading file: data\AEdata\AD\Admission Data FY2018-19DONE.xlsx
2022-07-20 09:30:03,442 [INFO] Processing file: data\AEdata\AD\Admission Data FY2018-19DONE.xlsx
2022-07-20 09:30:07,654 [INFO] Reading file: 

FileNotFoundError: [Errno 2] No such file or directory: 'data\\AEdata\\AD\\~$Admission Data FY2014-15DONE.xlsx'

# ICD-10 File

In [64]:
def process_icd10(xlsx):
    ICD10_3_Codes = (
        xlsx[
            [
                "Chapter_No",
                "Chapter_Desc",
                "Group_Code",
                "Group_Desc",
                "ICD10_3_Code",
                "ICD10_3_Code_Desc",
            ]
        ]
        .drop_duplicates("ICD10_3_Code")
        .set_index("ICD10_3_Code")
    )
    ICD10_Codes = xlsx[["ICD10_Code", "ICD10_3_Code", "WHO_Full_Desc"]].set_index(
        "ICD10_Code"
    )

    return ICD10_3_Codes, ICD10_Codes


if Notebook.RUN_ALL:
    infile = Notebook.DATA_DIR / "ICD10/ICD-10_MIT_2021_Excel_16-March_2021.xlsx"
    outfile = Notebook.DATA_DIR / "icd10.h5"

    logging.info(f"Reading file: {infile}")
    xlsx = pd.read_excel(infile, sheet_name="SA ICD-10 MIT 2021")

    logging.info(f"Processing file: {infile}")
    tc, c = process_icd10(xlsx)

    logging.info(f"Writing to: {outfile}")
    with pd.HDFStore(outfile) as store:
        store["ICD10_3_Codes"], store["ICD10_Codes"] = tc, c


In [124]:
if Notebook.RUN_ALL:
    infile = Notebook.DATA_DIR / "ICD10/Birkmeyer.csv"
    outfile = Notebook.DATA_DIR / "birkmeyer_icd10.h5"
    logging.info(f'Reading file: {infile}')
    df = pd.read_csv(infile)

    logging.info(f"Processing file: {infile}")
    #df = process_birkmeyer(df)

    logging.info(f"Writing to: {outfile}")
    df.to_hdf(outfile, 'table')

2022-07-19 08:44:53,357 [INFO] Reading file: data\ICD10\Birkmeyer.csv
2022-07-19 08:44:53,367 [INFO] Processing file: data\ICD10\Birkmeyer.csv
2022-07-19 08:44:53,367 [INFO] Writing to: data\birkmeyer_icd10.h5
